In [1]:
import mesa
import networkx as nx
import pandas as pd
import numpy as np
import threading
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mesa.visualization.modules import ChartModule, NetworkModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

In [2]:
cities = pd.read_csv("data/cities.csv")
demographic = pd.read_csv("data/demographic.csv")
turnout = pd.read_csv("data/turnout.csv")

In [3]:
threshold = 20 # Connect cities within this distance from each other

G = nx.Graph()
for _, row in cities.iterrows():
    G.add_node(row["city"], pos=(row["latitude"], row["longitude"]))

for city1 in G.nodes:
    for city2 in G.nodes:
        if city1 == city2:
            continue
        dist = np.linalg.norm(np.array(G.nodes[city1]["pos"]) - np.array(G.nodes[city2]["pos"]))

        if dist < threshold:
            print(f"Connected {city1} and {city2}")
            G.add_edge(city1, city2, weight=1/dist)

Connected manila and quezon city
Connected manila and caloocan
Connected manila and makati
Connected manila and marikina
Connected manila and parañaque
Connected manila and pasig
Connected manila and valenzuela
Connected manila and pasay
Connected manila and las piñas
Connected manila and mandaluyong
Connected manila and muntinlupa
Connected manila and malabon
Connected manila and navotas
Connected manila and san juan
Connected manila and taguig
Connected manila and pateros
Connected quezon city and manila
Connected quezon city and caloocan
Connected quezon city and makati
Connected quezon city and marikina
Connected quezon city and parañaque
Connected quezon city and pasig
Connected quezon city and valenzuela
Connected quezon city and pasay
Connected quezon city and las piñas
Connected quezon city and mandaluyong
Connected quezon city and muntinlupa
Connected quezon city and malabon
Connected quezon city and navotas
Connected quezon city and san juan
Connected quezon city and taguig
C

In [4]:
age_weight = {'0-4': 0, '5-9': 0, '10-14': 0, '15-19': 0.2, '20-24': 0.7, '25-29': 0.8, '30-34': 0.85, '35-39': 0.9, '40-44': 0.9, '45-49': 0.9, '50-54': 0.95, '55-59': 0.95, '60-64': 0.95, '65-69': 0.9, '70-74': 0.85, '75-79': 0.8, '80+': 0.7}
sex_weight = {'female': 1.1, 'male': 1.0}

self_weight = 0.7
same_city_weight = 0.15
nearby_weight = 0.15

total_population_by_city = demographic.groupby("city")["population"].sum()

class Agent(mesa.Agent):
    def __init__(self, id, model, city, age_group, sex):
        super().__init__(id, model)
        self.city = city
        self.age_group = age_group
        self.sex = sex
        city_turnout = turnout[turnout["city"] == city]
        self.turnout_prob = city_turnout["voted"].values[0] / city_turnout["registered"].values[0]
        self.turnout_prob *= self.demographic_adjustment()

        self.voted = False
    
    def demographic_adjustment(self):
        return age_weight[self.age_group] * sex_weight[self.sex]
    
    def step(self):
        if self.voted:
            return
        same_city_agents = self.model.grid.get_cell_list_contents([self.city]) # agents in the same city
        same_city_avg = np.mean([a.turnout_prob for a in same_city_agents if a != self])

        nearby_agents = []

        # Unweighted version
        # for neighbor_city in G.neighbors(self.city):
        #     nearby_agents += self.model.grid.get_cell_list_contents([neighbor_city])
        # nearby_avg = np.mean([a.turnout_prob for a in nearby_agents]) if nearby_agents else self.turnout_prob

        # Weighted version
        nearby_total = 0
        for neighbor_city in G.neighbors(self.city):
            weight = G[self.city][neighbor_city]["weight"] # based on the distance
            agents = self.model.grid.get_cell_list_contents([neighbor_city])

            for a in agents:
                nearby_agents.append(a.turnout_prob * weight)
                nearby_total += weight

        nearby_avg = sum(nearby_agents) / nearby_total if nearby_total > 0 else 0

        self.turnout_prob = ( # turnout probability is affected
            self_weight * self.turnout_prob + 
            same_city_weight * same_city_avg +
            nearby_weight * nearby_avg
        )
        
        
        if self.random.random() < self.turnout_prob:
            # print(self.city, "voted")
            self.voted = True


In [ ]:
scale = 10000

class Model(mesa.Model):
    def __init__(self):
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.NetworkGrid(G)
        self.datacollector = DataCollector(
            model_reporters=dict(
                # {"Total Voted": lambda m: [sum([1 for a in m.schedule.agents if a.voted])]}
                
                **{
                city: lambda m: [sum([1 for a in m.schedule.agents if (a.voted and a.city == city)])]
                for city in G.nodes
                }
            ),
            agent_reporters={
                "Voted": "voted"
            }
        )
        
        for _, row in demographic.iterrows():
            city = row["city"]
            age_group = row["age_group"]
            sex = row["sex"]
            
            # Assume proportion of those who register is equal to proportion in total population
            proportion = int(row["population"]) / total_population_by_city[city]
            count = turnout[turnout["city"] == city]["registered"].values[0] * proportion

            for i in range(int(count / scale)):
                agent = Agent(i, self, city, age_group, sex)
                self.schedule.add(agent)
                self.grid.place_agent(agent, city)
    
    def compute_avg_turnout(self):
        return np.mean([agent.turnout_prob for agent in self.schedule.agents])
    
    def count_voter_by_city(self, city):
        count = 0
        
        for agent in self.schedule.agents:
            if agent.voted:  # Only count if agent voted
                if agent.city == city:
                    count += 1
        
        return count
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        

In [6]:
model = Model()
for i in range(10):
    print(f"Step {i+1}")
    model.step()

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10


In [7]:
simulated_turnout = {}
for agent in model.schedule.agents:
    if agent.city not in simulated_turnout:
        simulated_turnout[agent.city] = 0
    if agent.voted:
        simulated_turnout[agent.city] += 1

actual_turnout = turnout.set_index("city")["voted"].to_dict()
common_cities = set(actual_turnout.keys()) & set(simulated_turnout.keys())
actual = [actual_turnout[city] for city in common_cities]
simulated = [simulated_turnout[city] * scale for city in common_cities]

comparison_df = pd.DataFrame({
    "City": list(common_cities),
    "Actual Voters": actual,
    "Simulated Voters": simulated
})

print(comparison_df.sort_values("City").reset_index(drop=True))

rmse = np.sqrt(mean_squared_error(actual, simulated))
mae = mean_absolute_error(actual, simulated)
print(f"Root Mean Squared Error: {rmse: .2f}")
print(f"Mean Absolute Error: {mae: .2f}")

           City  Actual Voters  Simulated Voters
0      caloocan         582521            550000
1     las piñas         242024            170000
2        makati         375103            290000
3       malabon         196218            130000
4   mandaluyong         184423             90000
5        manila         886133            970000
6      marikina         229267            140000
7    muntinlupa         252396            190000
8     parañaque         285438            190000
9         pasay         221411            160000
10        pasig         389419            330000
11  quezon city        1138511           1230000
12       taguig         371575            310000
13   valenzuela         363995            290000
Root Mean Squared Error:  75465.68
Mean Absolute Error:  73510.43


In [8]:
print(model.datacollector.get_model_vars_dataframe())

  manila quezon city caloocan makati marikina parañaque pasig valenzuela  \
0    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
1    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
2    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
3    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
4    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
5    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
6    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
7    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
8    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   
9    [0]         [0]      [0]    [0]      [0]       [0]   [0]        [0]   

  pasay las piñas mandaluyong muntinlupa malabon navotas san juan taguig  \
0   [0]       [0]         [0]        [0]     [0]     [0]      [0]    [0]   
1   [0]    

In [9]:
def network_portrayal(G):
    portrayal = {"nodes": [], "edges": []}
    for node_id in model.grid.nodes:
        agents = model.grid.nodes[node_id].get("agent", [])
        if not isinstance(agents, list):
            agents = [agents]
        voted = sum(1 for a in agents if a.voted)

        color = "green" if voted > 0 else "gray"
        portrayal["nodes"].append({"id": node_id, "color": color, "size": 10, "tooltip": f"{node_id}: {voted} voted"})
    for source, target in model.grid.edges:
        portrayal["edges"].append({"source": source, "target": target})
    return portrayal

# grid = NetworkModule(network_portrayal, 500, 500)



chart = ChartModule(
    # [{"Label": "Total Voted", "Color": "Black"}],
    [{"Label": "manila", "Color": "Black"},
     {"Label": "quezon city", "Color": "Black"},
     {"Label": "caloocan", "Color": "Black"},
     {"Label": "makati", "Color": "Black"},
     {"Label": "marikina", "Color": "Black"},
     {"Label": "parañaque", "Color": "Black"},
     {"Label": "pasig", "Color": "Black"},
     {"Label": "valenzuela", "Color": "Black"},
     {"Label": "pasay", "Color": "Black"},
     {"Label": "las piñas", "Color": "Black"},
     {"Label": "mandaluyong", "Color": "Black"},
     {"Label": "muntinlupa", "Color": "Black"},
     {"Label": "malabon", "Color": "Black"},
     {"Label": "navotas", "Color": "Black"},
     {"Label": "san juan", "Color": "Black"},
     {"Label": "taguig", "Color": "Black"},
     {"Label": "pateros", "Color": "Black"}
     ],
      data_collector_name="datacollector"
)

In [10]:
server = ModularServer(
    Model,
    [chart],
    "NCR Voter Turnout Model",
    {}
)
server.port = 4000

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


In [ ]:
server.launch()

Interface starting at http://127.0.0.1:4000


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
